Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  # here we add a regularization term
  logits = tf.matmul(tf_train_dataset, weights) + biases + 10**7*tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 1000

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 2.302585
Training accuracy: 8.8%
Validation accuracy: 19.3%
Loss at step 100: 0.645339
Training accuracy: 82.3%
Validation accuracy: 79.7%
Loss at step 200: 0.649495
Training accuracy: 82.4%
Validation accuracy: 79.4%
Loss at step 300: 0.618520
Training accuracy: 83.3%
Validation accuracy: 79.5%
Loss at step 400: 0.666209
Training accuracy: 82.1%
Validation accuracy: 79.2%
Loss at step 500: 0.645943
Training accuracy: 82.6%
Validation accuracy: 79.5%
Loss at step 600: 0.664865
Training accuracy: 81.9%
Validation accuracy: 79.1%
Loss at step 700: 0.650100
Training accuracy: 82.2%
Validation accuracy: 79.4%
Loss at step 800: 0.639792
Training accuracy: 82.4%
Validation accuracy: 79.4%
Loss at step 900: 0.656371
Training accuracy: 81.7%
Validation accuracy: 79.5%
Test accuracy: 86.3%


In [7]:
train_subset = 3000
batch_size = 64

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # here we add a regularization term
  logits = tf.matmul(tf_train_dataset, weights) + biases + 10**5*tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 5001
train_subset = 3000
train_subdataset = train_dataset[:train_subset]
train_sublabels = train_labels[:train_subset]
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subdataset[offset:(offset + batch_size), :]
    batch_labels = train_sublabels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.505713
Minibatch accuracy: 15.6%
Validation accuracy: 15.2%
Minibatch loss at step 500: 1.495437
Minibatch accuracy: 78.1%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 0.822082
Minibatch accuracy: 84.4%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.270778
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.164297
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.945974
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 1.226625
Minibatch accuracy: 79.7%
Validation accuracy: 79.7%
Minibatch loss at step 3500: 0.224784
Minibatch accuracy: 89.1%
Validation accuracy: 79.5%
Minibatch loss at step 4000: 0.750958
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Minibatch loss at step 4500: 1.190453
Minibatch accuracy: 76.6%
Validation accuracy: 79.5%
Minibatch loss at step 5000: 1.006905
Minibatch accuracy: 84.4%
Validation accura

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [40]:
batch_size = 128
HIDDEN_LAYER_SIZE = 2048
BETA = 1
graph = tf.Graph()
with graph.as_default():
    nn_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size*image_size))
    nn_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    #parameters 4 hidden layers
    nn_weights = tf.Variable(tf.truncated_normal([image_size*image_size, HIDDEN_LAYER_SIZE]))
    nn_bias = tf.Variable(tf.zeros([HIDDEN_LAYER_SIZE]))
    
    #hidden layer
    nn_fc = tf.nn.relu(tf.matmul(nn_train_dataset, nn_weights) + nn_bias + BETA*tf.nn.l2_loss(nn_weights))
    keep_prob = tf.placeholder(tf.float32)
    fc_dropout = tf.nn.dropout(nn_fc, keep_prob)
    
    #softmax layer
    s_weights = tf.Variable(tf.truncated_normal([HIDDEN_LAYER_SIZE, num_labels]))
    s_biases = tf.Variable(tf.zeros([num_labels]))
    
    nn_logits = tf.matmul(fc_dropout, s_weights) + s_biases + BETA*tf.nn.l2_loss(s_weights)
    y = tf.nn.softmax(nn_logits)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(nn_logits, nn_train_labels))
    optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(nn_train_labels,1))
    nn_accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    
    predict = tf.argmax(y, 1)

In [41]:
num_step = 10001
loss_set = []
train_accuracy_set = []
valid_accuracy_set = []
x_range = []
with tf.Session(graph=graph) as session:
    init = tf.initialize_all_variables()
    session.run(init)
    print('full connected neural network initialized!')
    print('the hidden layer has %d neurons'%HIDDEN_LAYER_SIZE)
    
    for step in range(num_step):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {nn_train_dataset:batch_data, nn_train_labels:batch_labels, keep_prob:0.5}
        _, l, predictions = session.run([optimizer, loss, predict], feed_dict = feed_dict)
        
        
        if(step % 10 == 0):
            nn_accu = nn_accuracy.eval(feed_dict = {nn_train_dataset:batch_data,
                                                               nn_train_labels: batch_labels,
                                                          keep_prob:1.0})
            valid_accuracy = nn_accuracy.eval(feed_dict = {nn_train_dataset:valid_dataset,
                                                               nn_train_labels: valid_labels,
                                                          keep_prob:1.0})
            train_accuracy_set.append(nn_accu)
            valid_accuracy_set.append(valid_accuracy)
            loss_set.append(l)
            x_range.append(step)
            if(step%1000 == 0):
                print('Minibatch loss at step %d => %.4f'%(step, l))
                print('Minibatch accuracy =>%.4f%%'%(nn_accu*100))
                print('Validation accuracy => %.4f%%'%(valid_accuracy*100))
                print('===========================================================')
    print('=====================================================')
    print('Accuracy on test dataset => %.4f%%'%(nn_accuracy.eval(feed_dict = {nn_train_dataset:test_dataset,
                                                               nn_train_labels: test_labels,
                                                                             keep_prob:1.0})*100))

full connected neural network initialized!
the hidden layer has 2048 neurons
Minibatch loss at step 0 => 49871460.0000
Minibatch accuracy =>10.9375%
Validation accuracy => 10.0000%
Minibatch loss at step 1000 => 3.4658
Minibatch accuracy =>78.9062%
Validation accuracy => 78.9000%
Minibatch loss at step 2000 => 0.4708
Minibatch accuracy =>89.8438%
Validation accuracy => 84.6700%
Minibatch loss at step 3000 => 0.5494
Minibatch accuracy =>89.8438%
Validation accuracy => 85.6500%
Minibatch loss at step 4000 => 0.5719
Minibatch accuracy =>85.1562%
Validation accuracy => 86.5200%
Minibatch loss at step 5000 => 0.6228
Minibatch accuracy =>89.0625%
Validation accuracy => 86.7200%
Minibatch loss at step 6000 => 0.6156
Minibatch accuracy =>87.5000%
Validation accuracy => 86.9800%
Minibatch loss at step 7000 => 0.5618
Minibatch accuracy =>89.8438%
Validation accuracy => 87.0100%
Minibatch loss at step 8000 => 0.5080
Minibatch accuracy =>90.6250%
Validation accuracy => 87.1300%
Minibatch loss at s

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
